In [ ]:
from py123d.datatypes.scene.arrow.arrow_scene_builder import ArrowSceneBuilder
from py123d.datatypes.scene.scene_filter import SceneFilter

from py123d.common.multithreading.worker_sequential import Sequential
from py123d.datatypes.sensors.pinhole_camera import PinholeCameraType

In [ ]:
# splits = ["wopd_val"]
# splits = ["carla_test"]
# splits = ["nuplan-mini_test"]
# splits = ["av2-sensor-mini_train"]


splits = ["pandaset_train"]
# log_names = None


log_names = None
scene_uuids = None

scene_filter = SceneFilter(
    split_names=splits,
    log_names=log_names,
    scene_uuids=scene_uuids,
    duration_s=6.0,
    history_s=0.0,
    timestamp_threshold_s=20,
    shuffle=True,
    pinhole_camera_types=[PinholeCameraType.PCAM_F0],
)
scene_builder = ArrowSceneBuilder()
worker = Sequential()
# worker = RayDistributed()
scenes = scene_builder.get_scenes(scene_filter, worker)

print(f"Found {len(scenes)} scenes")

In [ ]:
from matplotlib import pyplot as plt
from py123d.datatypes.scene.abstract_scene import AbstractScene
from py123d.visualization.matplotlib.camera import add_box_detections_to_camera_ax, add_camera_ax

iteration = 0
scene = scenes[0]

scene: AbstractScene
print(scene.uuid, scene.available_pinhole_camera_types)

scale = 3.0
fig, ax = plt.subplots(2, 3, figsize=(scale * 6, scale * 2.5))


camera_ax_mapping = {
    PinholeCameraType.PCAM_L0: ax[0, 0],
    PinholeCameraType.PCAM_F0: ax[0, 1],
    PinholeCameraType.PCAM_R0: ax[0, 2],
    PinholeCameraType.PCAM_L1: ax[1, 0],
    PinholeCameraType.PCAM_B0: ax[1, 1],
    PinholeCameraType.PCAM_R1: ax[1, 2],
}


for camera_type, ax_ in camera_ax_mapping.items():
    camera = scene.get_pinhole_camera_at_iteration(iteration, camera_type)
    box_detections = scene.get_box_detections_at_iteration(iteration)
    ego_state = scene.get_ego_state_at_iteration(iteration)

    add_box_detections_to_camera_ax(
        ax_,
        camera,
        box_detections,
        ego_state,
    )
    ax_.set_title(f"Camera: {camera_type.name}")